In [14]:
import cv2
import numpy as np
from time import sleep

width_min=80 
height_min=80 

offset=6
 

delay= 60 

detect = []
vehicle_count = 0

def page_center(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx,cy

cap = cv2.VideoCapture('night_video.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
   
size = (frame_width, frame_height)
   
result = cv2.VideoWriter('output_night.avi', cv2.VideoWriter_fourcc(*'MJPG'),10, size)
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

while True:
    ret , frame1 = cap.read()
    height, width = frame1.shape[0:2]
    time = float(1/delay)
    sleep(time) 
    grey = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey,(3,3),5)
    img_sub = bg_subtractor.apply(blur)
    dilated_img = cv2.dilate(img_sub,np.ones((5,5)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    dilated = cv2.morphologyEx (dilated_img, cv2. MORPH_CLOSE , kernel)
    dilated = cv2.morphologyEx (dilated, cv2. MORPH_CLOSE , kernel)
    contours,h=cv2.findContours(dilated,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    cv2.line(frame1, (0, height-100), (width, height-100), (255,127,0), 3) 
    for(i,c) in enumerate(contours):
        (x,y,w,h) = cv2.boundingRect(c)
        line_cord = height-100
        validate_contour = (w >= width_min) and (h >= height_min)
        if not validate_contour:
            continue

        cv2.rectangle(frame1,(x,y),(x+w,y+h),(0,255,0),2)        
        center = page_center(x, y, w, h)
        detect.append(center)
        cv2.circle(frame1, center, 4, (0, 0,255), -1)

        for (x,y) in detect:
            if y<(line_cord+offset) and y>(line_cord-offset):
                vehicle_count+=1
                cv2.line(frame1, (0, height-100), (width, height-100), (0,127,255), 3)  
                detect.remove((x,y))
                print("No. of vehicle is detected : "+str(vehicle_count))        
       
    cv2.putText(frame1, "VEHICLE COUNT : "+str(vehicle_count), (0, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255),2)
    result.write(frame1)
    cv2.imshow("Video Original" , frame1)
    cv2.imshow("Detectar",dilated)



    if cv2.waitKey(1) == ord('a'):
        print ("pressed quit")
        break
   
cv2.destroyAllWindows()
cap.release()

No. of vehicle is detected : 1
No. of vehicle is detected : 2
No. of vehicle is detected : 3
No. of vehicle is detected : 4
No. of vehicle is detected : 5
No. of vehicle is detected : 6
No. of vehicle is detected : 7
No. of vehicle is detected : 8
No. of vehicle is detected : 9
No. of vehicle is detected : 10
No. of vehicle is detected : 11
No. of vehicle is detected : 12
No. of vehicle is detected : 13
No. of vehicle is detected : 14
pressed quit
